In [8]:
import gtfs_kit as gk

# 载入GTFS数据
feed = gk.read_feed('gtfs.zip', dist_units='m')


In [38]:
import gtfs_kit as gk
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString

# 载入GTFS数据
feed = gk.read_feed('gtfs.zip', dist_units='m')

# 将站点数据转换为 GeoDataFrame
stops_geo = gpd.GeoDataFrame(
    feed.stops,
    geometry=gpd.points_from_xy(feed.stops.stop_lon, feed.stops.stop_lat),
    crs="EPSG:4326"
)


In [39]:
import geopandas as gpd
from shapely.geometry import Point

# 假设A点坐标
point_a = Point(144.955482,-37.820819)
point_b = Point(144.964422,-37.809364)

# 创建地理数据框
stations = gpd.GeoDataFrame(feed.stops, geometry=gpd.points_from_xy(feed.stops.stop_lon, feed.stops.stop_lat))
def calculate_nearest_station(point):
    stations['distance'] = stations.geometry.distance(point)
    nearest_station = stations.loc[stations['distance'].idxmin()]
    return nearest_station
station_a=calculate_nearest_station(point_a)
station_b=calculate_nearest_station(point_b)



In [61]:
station_a

stop_id                                          21232
stop_name    1-Spencer St/Flinders St (Melbourne City)
stop_lat                                    -37.821051
stop_lon                                    144.955381
geometry      POINT (144.95538078963 -37.821051394367)
distance                                      0.000253
Name: 1338, dtype: object

In [41]:
# 分析经过每个站点的路线
routes_at_stops = feed.stop_times.merge(feed.trips[['trip_id', 'route_id']], on='trip_id')
routes_at_stops = routes_at_stops.groupby('stop_id')['route_id'].apply(set)

# 查找直达路线和换乘点
routes_from_a = routes_at_stops[station_a['stop_id']]
routes_to_b = routes_at_stops[station_b['stop_id']]

# 寻找直达路线
direct_routes = routes_from_a.intersection(routes_to_b)

# 如果没有直达路线，寻找可能的换乘点
if not direct_routes:
    transfer_stations = []
    for stop_id, routes in routes_at_stops.items():
        if routes_from_a.intersection(routes) and routes_to_b.intersection(routes):
            transfer_stations.append(stop_id)


In [71]:
a_station_id = station_a.stop_id
b_station_id = station_b.stop_id

a_trips = set(feed.stop_times[feed.stop_times['stop_id'] == a_station_id]['trip_id'])
b_trips = set(feed.stop_times[feed.stop_times['stop_id'] == b_station_id]['trip_id'])

# 直达路线的行程 ID
direct_trip_ids = a_trips.intersection(b_trips)

# 初始化最短行程变量
min_duration = float('inf')
shortest_trip = None

# 遍历直达行程
for trip_id in direct_trip_ids:
    trip_stops = feed.stop_times[feed.stop_times['trip_id'] == trip_id].sort_values('stop_sequence')
    start_index = trip_stops[trip_stops['stop_id'] == a_station_id].index.min()
    end_index = trip_stops[trip_stops['stop_id'] == b_station_id].index.min()
    
    # 确保方向正确，取决于 start_index 和 end_index 的关系
    if start_index < end_index:  # 正向行程
        relevant_stops = trip_stops.loc[start_index:end_index]
    elif start_index > end_index:  # 反向行程
        relevant_stops = trip_stops.loc[end_index:start_index]
    else:
        continue  # 如果起点和终点相同

    # 计算行程总持续时间
    departure_time = pd.to_timedelta(relevant_stops.iloc[0]['departure_time'])
    arrival_time = pd.to_timedelta(relevant_stops.iloc[-1]['arrival_time'])
    duration = (arrival_time - departure_time).total_seconds()

    # 更新最短行程
    if duration < min_duration:
        min_duration = duration
        shortest_trip = trip_id, relevant_stops

# 如果找到最短行程
if shortest_trip:
    trip_id, stops, duration = shortest_trip
    route_id = feed.trips[feed.trips['trip_id'] == trip_id]['route_id'].iloc[0]
    print(f"最短直达路线行程ID: {trip_id}, 持续时间: {duration/60} 分钟, 路线ID: {route_id}")
    for _, stop in stops.iterrows():
        stop_name = feed.stops[feed.stops['stop_id'] == stop['stop_id']].iloc[0]['stop_name']
        print(f"{stop['stop_id']} - {stop_name}")
else:
    print("没有找到直达路线，需要考虑换乘")

ValueError: not enough values to unpack (expected 3, got 2)

In [72]:
def get_route_id_from_trip(feed, trip_id):
    # 从 trips 表中查询指定 trip_id 的 route_id
    route_id = feed.trips[feed.trips['trip_id'] == trip_id]['route_id'].iloc[0]
    return route_id

# 示例：假设你已知的 trip_id
trip_id = "2.T0.3-35-mjp-4.1.H"

# 调用函数，获取 route_id
route_id = get_route_id_from_trip(feed, trip_id)
print(f"对应的 route_id 为: {route_id}")


对应的 route_id 为: 3-35-mjp-4


In [59]:
station_b

stop_id                                           18046
stop_name    6-Swanston St/La Trobe St (Melbourne City)
stop_lat                                     -37.809341
stop_lon                                     144.964484
geometry     POINT (144.964484165349 -37.8093406405031)
distance                                       0.000066
Name: 226, dtype: object

In [112]:
import pandas as pd
import datetime
from datetime import timedelta
import geopandas as gpd
from shapely.geometry import Point
import gtfs_realtime_pb2
import requests

# 载入GTFS数据
feed = gk.read_feed('gtfs.zip', dist_units='m')

# 假设A点坐标
point_a = Point(144.962457,-37.807190)
point_b = Point(144.954063,-37.817783)

# 创建地理数据框
stations = gpd.GeoDataFrame(feed.stops, geometry=gpd.points_from_xy(feed.stops.stop_lon, feed.stops.stop_lat))
def calculate_nearest_station(point):
    stations['distance'] = stations.geometry.distance(point)
    nearest_station = stations.loc[stations['distance'].idxmin()]
    return nearest_station
station_a=calculate_nearest_station(point_a)
station_b=calculate_nearest_station(point_b)
a_station_id = station_a.stop_id
b_station_id = station_b.stop_id


def parse_vehicle_data_for_trip_id(data, trip_id_to_find):
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(data)

    inaccessible_models = ['Z1', 'Z3', 'A1', 'A2']
    for entity in feed.entity:
        if entity.HasField('vehicle'):
            vehicle = entity.vehicle
            trip = vehicle.trip
            if trip.trip_id == trip_id_to_find:  # Check if the current trip ID matches the desired trip ID
                position = vehicle.position
                vehicle_details = vehicle.vehicle
                # Check if the vehicle label contains any of the specified inaccessible models
                accessible = 0 if any(model in vehicle_details.label for model in inaccessible_models) else 1
                # vehicle_info = {
                #     "Vehicle ID": vehicle_details.id,
                #     "Vehicle Label": vehicle_details.label,
                #     "Accessible": accessible  # Add accessibility info
                # }
                return accessible  # Return the found vehicle info immediately
    return 1  # Return None if no vehicle matches the specified trip ID

# Get data
url = "https://data-exchange-api.vicroads.vic.gov.au/opendata/gtfsr/v1/tram/vehicleposition"
headers = {
    "Ocp-Apim-Subscription-Key": "af02fa9cf4764385ad592b462db4ec8e"
}
response = requests.get(url, headers=headers)




def parse_time(t):
    if pd.isna(t):
        return None
    hours, minutes, seconds = map(int, t.split(':'))
    return timedelta(hours=hours, minutes=minutes, seconds=seconds)

def find_next_departures(feed, a_station_id, b_station_id, hours_offset=15):
    now = datetime.datetime.now() - timedelta(hours=hours_offset)
    now_time = now.time()
    now_delta = timedelta(hours=now_time.hour, minutes=now_time.minute, seconds=now_time.second)
    print(f"Adjusted Current Time Delta: {now_delta}")

    a_trips = set(feed.stop_times[feed.stop_times['stop_id'] == a_station_id]['trip_id'])
    b_trips = set(feed.stop_times[feed.stop_times['stop_id'] == b_station_id]['trip_id'])
    direct_trip_ids = a_trips.intersection(b_trips)

    min_duration = float('inf')
    shortest_trip = None

    for trip_id in direct_trip_ids:
        trip_stops = feed.stop_times[feed.stop_times['trip_id'] == trip_id].sort_values('stop_sequence')
        start_index = trip_stops[trip_stops['stop_id'] == a_station_id].index.min()
        end_index = trip_stops[trip_stops['stop_id'] == b_station_id].index.min()
        accessible = parse_vehicle_data_for_trip_id(response.content, trip_id)
        if not accessible or accessible != 1:

            continue
        

        if pd.isna(start_index) or pd.isna(end_index):
            print("Skipping trip due to NA index.")
            continue

        if start_index < end_index:
            relevant_stops = trip_stops.loc[start_index:end_index]
        elif start_index > end_index:
            relevant_stops = trip_stops.loc[end_index:start_index]
        else:
            continue

        if not relevant_stops.empty:
            departure_time_str = relevant_stops.iloc[0]['departure_time']
            departure_time = parse_time(departure_time_str)
            arrival_time_str = relevant_stops.iloc[-1]['arrival_time']
            arrival_time = parse_time(arrival_time_str)

            if departure_time and arrival_time and departure_time > now_delta:
                duration = (arrival_time - departure_time).total_seconds()
                if duration < min_duration:
                    min_duration = duration
                    shortest_trip = trip_id, departure_time_str, arrival_time_str, duration / 60,relevant_stops  # Storing more info here

    return shortest_trip

# Example usage

nearest_trip = find_next_departures(feed, a_station_id, b_station_id)
if nearest_trip:
    trip_id, departure_time_str, arrival_time_str, duration_minutes, stops = nearest_trip
    print(f"Next departure trip ID: {trip_id}, Departure at: {departure_time_str}, Arrives at: {arrival_time_str}, Duration: {duration_minutes} minutes")
    for _, stop in stops.iterrows():
        stop_name = feed.stops[feed.stops['stop_id'] == stop['stop_id']].iloc[0]['stop_name']
        print(f"{stop['stop_id']} - {stop_name}")
else:
    print("No upcoming trips found")


Adjusted Current Time Delta: 12:20:58
No upcoming trips found


In [106]:
import datetime
from datetime import timedelta
import pandas as pd
import gtfs_kit as gk
import geopandas as gpd
from shapely.geometry import Point, LineString

import geopandas as gpd
from shapely.geometry import Point

# 载入GTFS数据
feed = gk.read_feed('gtfs.zip', dist_units='m')

def find_next_departures(feed, a_station_id, b_station_id, hours_offset=15):
    now = datetime.datetime.now() - timedelta(hours=hours_offset)
    now_time = now.time()
    now_delta = timedelta(hours=now_time.hour, minutes=now_time.minute, seconds=now_time.second)
    print(f"Adjusted Current Time Delta: {now_delta}")

    a_trips = set(feed.stop_times[feed.stop_times['stop_id'] == a_station_id]['trip_id'])
    b_trips = set(feed.stop_times[feed.stop_times['stop_id'] == b_station_id]['trip_id'])
    direct_trip_ids = a_trips.intersection(b_trips)

    min_duration = float('inf')
    shortest_trip = None

    if not direct_trip_ids:
        # If no direct trip, find the nearest stop to b_station_id
        return find_nearest_stop(feed, a_station_id, b_station_id)

    for trip_id in direct_trip_ids:
        trip_stops = feed.stop_times[feed.stop_times['trip_id'] == trip_id].sort_values('stop_sequence')
        start_index = trip_stops[trip_stops['stop_id'] == a_station_id].index.min()
        end_index = trip_stops[trip_stops['stop_id'] == b_station_id].index.min()

        if pd.isna(start_index) or pd.isna(end_index):
            continue

        relevant_stops = trip_stops.loc[start_index:end_index]
        departure_time_str = relevant_stops.iloc[0]['departure_time']
        departure_time = parse_time(departure_time_str)
        arrival_time_str = relevant_stops.iloc[-1]['arrival_time']
        arrival_time = parse_time(arrival_time_str)

        if departure_time > now_delta:
            duration = (arrival_time - departure_time).total_seconds()
            if duration < min_duration:
                min_duration = duration
                shortest_trip = trip_id, departure_time_str, arrival_time_str, duration / 60

    return shortest_trip





import geopandas as gpd
from shapely.geometry import Point

def find_nearest_stop(feed, a_station_id, b_station_id):
    # Convert stops to a GeoDataFrame with geometry
    stops_geo = gpd.GeoDataFrame(
        feed.stops,
        geometry=gpd.points_from_xy(feed.stops.stop_lon, feed.stops.stop_lat),
        crs='EPSG:4326'
    )

    # Project to a suitable CRS for distance calculation (e.g., a UTM zone)
    # You'll need to choose a UTM zone that's appropriate for your area
    stops_geo = stops_geo.to_crs(epsg=32633)  # Example UTM zone, adjust as necessary

    # Get geometry for both start (a_station) and end (b_station)
    a_point = stops_geo.loc[stops_geo['stop_id'] == a_station_id, 'geometry'].squeeze()
    b_point = stops_geo.loc[stops_geo['stop_id'] == b_station_id, 'geometry'].squeeze()

    # Use spatial index to find the nearest stop to b_point
    nearest_indices = stops_geo.sindex.nearest(b_point, return_all=True)
    potential_nearest_stops = stops_geo.iloc[list(nearest_indices)].copy()

    # Calculate actual distances to these stops and find the nearest
    potential_nearest_stops['distance'] = potential_nearest_stops.geometry.distance(b_point)
    nearest_stop_id = potential_nearest_stops.loc[potential_nearest_stops['distance'].idxmin(), 'stop_id']

    # Find trips from a_station_id that include the nearest_stop_id
    trips_from_a = set(feed.stop_times[feed.stop_times['stop_id'] == a_station_id]['trip_id'])
    trips_including_nearest = set(feed.stop_times[feed.stop_times['stop_id'] == nearest_stop_id]['trip_id'])
    valid_trips = trips_from_a.intersection(trips_including_nearest)

    if valid_trips:
        trip_id = valid_trips.pop()  # Get the first valid trip_id
        
        # Retrieve stops from a_station_id to the nearest_stop_id
        trip_stops = feed.stop_times[feed.stop_times['trip_id'] == trip_id].merge(feed.stops, on='stop_id', how='left')
        trip_stops = trip_stops.sort_values('stop_sequence')
        start_index = trip_stops[trip_stops['stop_id'] == a_station_id].index.min()
        end_index = trip_stops[trip_stops['stop_id'] == nearest_stop_id].index.min()
        
        if start_index > end_index:
            trip_stops = trip_stops.loc[end_index:start_index]
        else:
            trip_stops = trip_stops.loc[start_index:end_index]
        
        print("Trip ID:", trip_id)
        print("Stops for this trip from", a_station_id, "to nearest stop to", b_station_id, ":")
        print(trip_stops[['stop_name', 'stop_id', 'stop_sequence']])
        
        return trip_id, trip_stops
    return None, None





def parse_time(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    return datetime.timedelta(hours=hours, minutes=minutes, seconds=seconds)


# 假设A点坐标
point_a = Point(144.962457,-37.807190)
point_b = Point(144.963055,-37.818782)

# 创建地理数据框
stations = gpd.GeoDataFrame(feed.stops, geometry=gpd.points_from_xy(feed.stops.stop_lon, feed.stops.stop_lat))
def calculate_nearest_station(point):
    stations['distance'] = stations.geometry.distance(point)
    nearest_station = stations.loc[stations['distance'].idxmin()]
    return nearest_station
station_a=calculate_nearest_station(point_a)
station_b=calculate_nearest_station(point_b)
a_station_id = station_a.stop_id
b_station_id = station_b.stop_id


nearest_trip = find_next_departures(feed, a_station_id, b_station_id)



Adjusted Current Time Delta: 12:09:05


ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [101]:
nearest_stop_id

NameError: name 'nearest_stop_id' is not defined

In [ ]:
nearest_trip

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 假设 'feed' 是你已经加载了 GTFS 数据的对象
# 读取 stops 数据
stops_df = pd.read_csv('stops.txt')
stops_df = stops_df[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]

# 创建地理数据框，以便操作地理空间数据
gdf_stops = gpd.GeoDataFrame(
    stops_df,
    geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat)
)

# 根据 trip_id 读取行程中站台的顺序，这里假设你有 trip_id
trip_id = '199.T0.3-3-mjp-4.3.R'
stop_times_df = pd.read_csv('stop_times.txt')
route_stops = stop_times_df[stop_times_df['trip_id'] == trip_id].sort_values('stop_sequence')

# 合并以获取完整的站台信息
route_stops_with_info = route_stops.merge(stops_df, on='stop_id')

# 提取路线坐标和站台名称
route_coordinates = route_stops_with_info[['stop_lon', 'stop_lat']].values.tolist()
stop_names = route_stops_with_info['stop_name'].tolist()

# 输出结果
print("Route Coordinates:")
for coord in route_coordinates:
    print(coord)

print("\nStop Names:")
for name in stop_names:
    print(name)


In [3]:
import pandas as pd
import datetime
from datetime import timedelta
import geopandas as gpd
from shapely.geometry import Point
import gtfs_realtime_pb2
import gtfs_kit as gk
import requests

# 载入GTFS数据
feed = gk.read_feed('gtfs.zip', dist_units='m')

# 假设A点坐标
point_a = Point(144.962457,-37.807190)
point_b = Point(144.954063,-37.817783)

# 创建地理数据框
stations = gpd.GeoDataFrame(feed.stops, geometry=gpd.points_from_xy(feed.stops.stop_lon, feed.stops.stop_lat))
def calculate_nearest_station(point):
    stations['distance'] = stations.geometry.distance(point)
    nearest_station = stations.loc[stations['distance'].idxmin()]
    return nearest_station
station_a=calculate_nearest_station(point_a)
station_b=calculate_nearest_station(point_b)

def find_alternative_route(feed, a_station_id, point_b, max_distance=1000):
    # 为所有站点计算到B点的距离
    stations['distance_to_b'] = stations.geometry.distance(point_b)
    
    # 筛选出距离B点在max_distance范围内的站点
    relevant_stations = stations[stations['distance_to_b'] <= max_distance]
    
    if relevant_stations.empty:
        print("No stations within 1 km of point B.")
        return None, None

    # 找出从A点出发的车次所经过的所有站点
    a_trips = set(feed.stop_times[feed.stop_times['stop_id'] == a_station_id]['trip_id'])
    relevant_stops = feed.stop_times[feed.stop_times['trip_id'].isin(a_trips)]

    # 筛选出符合条件的站点
    potential_stations = relevant_stations[relevant_stations['stop_id'].isin(relevant_stops['stop_id'])]
    if potential_stations.empty:
        print("No relevant stops found.")
        return None, None

    nearest_station_to_b = potential_stations.sort_values('distance_to_b').iloc[0]
    nearest_station_id = nearest_station_to_b['stop_id']
    
    # 查找最佳车次
    
    return  nearest_station_to_b

# 调用函数
a_station_id = station_a.stop_id

nearest_station_to_b = find_alternative_route(feed, a_station_id, point_b, 1000)
b_station_id = nearest_station_to_b.stop_id



def parse_vehicle_data_for_trip_id(data, trip_id_to_find):
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(data)

    inaccessible_models = ['Z1', 'Z3', 'A1', 'A2']
    for entity in feed.entity:
        if entity.HasField('vehicle'):
            vehicle = entity.vehicle
            trip = vehicle.trip
            if trip.trip_id == trip_id_to_find:  # Check if the current trip ID matches the desired trip ID
                position = vehicle.position
                vehicle_details = vehicle.vehicle
                # Check if the vehicle label contains any of the specified inaccessible models
                accessible = 0 if any(model in vehicle_details.label for model in inaccessible_models) else 1
                # vehicle_info = {
                #     "Vehicle ID": vehicle_details.id,
                #     "Vehicle Label": vehicle_details.label,
                #     "Accessible": accessible  # Add accessibility info
                # }
                return accessible  # Return the found vehicle info immediately
    return 1  # Return None if no vehicle matches the specified trip ID

# Get data
url = "https://data-exchange-api.vicroads.vic.gov.au/opendata/gtfsr/v1/tram/vehicleposition"
headers = {
    "Ocp-Apim-Subscription-Key": "af02fa9cf4764385ad592b462db4ec8e"
}
response = requests.get(url, headers=headers)




def parse_time(t):
    if pd.isna(t):
        return None
    hours, minutes, seconds = map(int, t.split(':'))
    return timedelta(hours=hours, minutes=minutes, seconds=seconds)

def find_next_departures(feed, a_station_id, b_station_id, hours_offset=15):
    now = datetime.datetime.now() - timedelta(hours=hours_offset)
    now_time = now.time()
    now_delta = timedelta(hours=now_time.hour, minutes=now_time.minute, seconds=now_time.second)
    print(f"Adjusted Current Time Delta: {now_delta}")

    a_trips = set(feed.stop_times[feed.stop_times['stop_id'] == a_station_id]['trip_id'])
    b_trips = set(feed.stop_times[feed.stop_times['stop_id'] == b_station_id]['trip_id'])
    direct_trip_ids = a_trips.intersection(b_trips)

    min_duration = float('inf')
    shortest_trip = None

    for trip_id in direct_trip_ids:
        trip_stops = feed.stop_times[feed.stop_times['trip_id'] == trip_id].sort_values('stop_sequence')
        start_index = trip_stops[trip_stops['stop_id'] == a_station_id].index.min()
        end_index = trip_stops[trip_stops['stop_id'] == b_station_id].index.min()
        accessible = parse_vehicle_data_for_trip_id(response.content, trip_id)
        if not accessible or accessible != 1:

            continue
        

        if pd.isna(start_index) or pd.isna(end_index):
            print("Skipping trip due to NA index.")
            continue

        if start_index < end_index:
            relevant_stops = trip_stops.loc[start_index:end_index]
        elif start_index > end_index:
            relevant_stops = trip_stops.loc[end_index:start_index]
        else:
            continue

        if not relevant_stops.empty:
            departure_time_str = relevant_stops.iloc[0]['departure_time']
            departure_time = parse_time(departure_time_str)
            arrival_time_str = relevant_stops.iloc[-1]['arrival_time']
            arrival_time = parse_time(arrival_time_str)

            if departure_time and arrival_time and departure_time > now_delta:
                duration = (arrival_time - departure_time).total_seconds()
                if duration < min_duration:
                    min_duration = duration
                    shortest_trip = trip_id, departure_time_str, arrival_time_str, duration / 60,relevant_stops  # Storing more info here

    return shortest_trip

# Example usage

nearest_trip = find_next_departures(feed, a_station_id, b_station_id)
if nearest_trip:
    trip_id, departure_time_str, arrival_time_str, duration_minutes, stops = nearest_trip
    print(f"Next departure trip ID: {trip_id}, Departure at: {departure_time_str}, Arrives at: {arrival_time_str}, Duration: {duration_minutes} minutes")
    for _, stop in stops.iterrows():
        stop_name = feed.stops[feed.stops['stop_id'] == stop['stop_id']].iloc[0]['stop_name']
        print(f"{stop['stop_id']} - {stop_name}")
else:
    print("No upcoming trips found")


Adjusted Current Time Delta: 13:21:57


KeyboardInterrupt: 

In [5]:
nearest_trip

NameError: name 'nearest_trip' is not defined

In [12]:
import pandas as pd
import datetime
from datetime import timedelta
import geopandas as gpd
from shapely.geometry import Point
import gtfs_realtime_pb2
import requests

# 载入GTFS数据
feed = gk.read_feed('gtfs.zip', dist_units='m')

# 假设A点坐标
point_a = Point(144.962457,-37.807190)
point_b = Point(144.954063,-37.817783)

# 创建地理数据框
stations = gpd.GeoDataFrame(feed.stops, geometry=gpd.points_from_xy(feed.stops.stop_lon, feed.stops.stop_lat))
def calculate_nearest_station(point):
    stations['distance'] = stations.geometry.distance(point)
    nearest_station = stations.loc[stations['distance'].idxmin()]
    return nearest_station
station_a=calculate_nearest_station(point_a)
station_b=calculate_nearest_station(point_b)
a_station_id = station_a.stop_id
b_station_id = station_b.stop_id


def parse_vehicle_data_for_trip_id(data, trip_id_to_find):
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(data)

    inaccessible_models = ['Z1', 'Z3', 'A1', 'A2']
    for entity in feed.entity:
        if entity.HasField('vehicle'):
            vehicle = entity.vehicle
            trip = vehicle.trip
            if trip.trip_id == trip_id_to_find:  # Check if the current trip ID matches the desired trip ID
                position = vehicle.position
                vehicle_details = vehicle.vehicle
                # Check if the vehicle label contains any of the specified inaccessible models
                accessible = 0 if any(model in vehicle_details.label for model in inaccessible_models) else 1
                # vehicle_info = {
                #     "Vehicle ID": vehicle_details.id,
                #     "Vehicle Label": vehicle_details.label,
                #     "Accessible": accessible  # Add accessibility info
                # }
                return accessible  # Return the found vehicle info immediately
    return 1  # Return None if no vehicle matches the specified trip ID

# Get data
url = "https://data-exchange-api.vicroads.vic.gov.au/opendata/gtfsr/v1/tram/vehicleposition"
headers = {
    "Ocp-Apim-Subscription-Key": "af02fa9cf4764385ad592b462db4ec8e"
}
response = requests.get(url, headers=headers)




def parse_time(t):
    if pd.isna(t):
        return None
    hours, minutes, seconds = map(int, t.split(':'))
    return timedelta(hours=hours, minutes=minutes, seconds=seconds)

def find_next_departures(feed, a_station_id, b_station_id, hours_offset=10):
    now = datetime.datetime.now() - timedelta(hours=hours_offset)
    now_time = now.time()
    now_delta = timedelta(hours=now_time.hour, minutes=now_time.minute, seconds=now_time.second)
    print(f"Adjusted Current Time Delta: {now_delta}")

    # 筛选出相关车次
    a_trips = set(feed.stop_times[feed.stop_times['stop_id'] == a_station_id]['trip_id'])
    b_trips = set(feed.stop_times[feed.stop_times['stop_id'] == b_station_id]['trip_id'])
    direct_trip_ids = a_trips.intersection(b_trips)

    min_duration = float('inf')
    shortest_trip = None

    # 避免在循环内排序
    trip_stops_data = feed.stop_times[feed.stop_times['trip_id'].isin(direct_trip_ids)]
    trip_stops_data_sorted = trip_stops_data.sort_values(['trip_id', 'stop_sequence'])

    for trip_id in direct_trip_ids:
    trip_stops = trip_stops_data_sorted[trip_stops_data_sorted['trip_id'] == trip_id]
    start_index = trip_stops[trip_stops['stop_id'] == a_station_id].index.min()
    end_index = trip_stops[trip_stops['stop_id'] == b_station_id].index.min()
    accessible = parse_vehicle_data_for_trip_id(response.content, trip_id)
    if not accessible:
        continue

    if pd.isna(start_index) or pd.isna(end_index):
        continue

    relevant_stops = trip_stops.iloc[min(start_index, end_index):max(start_index, end_index) + 1]

    # 检查 relevant_stops 是否为空
    if relevant_stops.empty:
        print(f"No stops found for trip ID {trip_id} between specified stations.")
        continue

    departure_time_str = relevant_stops.iloc[0]['departure_time']
    arrival_time_str = relevant_stops.iloc[-1]['arrival_time']

    departure_time = parse_time(departure_time_str)
    arrival_time = parse_time(arrival_time_str)

    if departure_time and arrival_time and departure_time > now_delta:
        duration = (arrival_time - departure_time).total_seconds()
        if duration < min_duration:
            min_duration = duration
            shortest_trip = (trip_id, departure_time_str, arrival_time_str, duration / 60, accessible)


    return shortest_trip

# Example usage

nearest_trip = find_next_departures(feed, a_station_id, b_station_id)
if nearest_trip:
    trip_id, departure_time_str, arrival_time_str, duration_minutes, stops = nearest_trip
    print(f"Next departure trip ID: {trip_id}, Departure at: {departure_time_str}, Arrives at: {arrival_time_str}, Duration: {duration_minutes} minutes")
    for _, stop in stops.iterrows():
        stop_name = feed.stops[feed.stops['stop_id'] == stop['stop_id']].iloc[0]['stop_name']
        print(f"{stop['stop_id']} - {stop_name}")
else:
    print("No upcoming trips found")


Adjusted Current Time Delta: 18:29:53
No upcoming trips found


In [13]:
b_station_id = nearest_station_to_b.stop_id

In [14]:
find_next_departures(feed, a_station_id, b_station_id)

Adjusted Current Time Delta: 18:29:59


IndexError: single positional indexer is out-of-bounds